In [1]:
using Laplacians

objc[2414]: Class TKApplication is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.
objc[2414]: Class TKMenu is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.
objc[2414]: Class TKContentView is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.
objc[2414]: Class TKWindow is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.


In [29]:
include("treeSolver.jl")
include("IO.jl")
include("generators.jl")
include("toHighPrecision.jl")
include("verbosePCG.jl")

pcgVBLASMatNorm (generic function with 1 method)

In [19]:
pwd()

"/Users/serbanstan/git/TreePCG/julia"

In [20]:
a = readFromFile("../graphs/se_1/graph.mtx");

In [21]:
tree = readFromFile("../graphs/se_1/tree.mtx");

In [22]:
x = readFromFile("../graphs/se_1/x.vec");

In [23]:
# b = readFromFile("../graphs/se_1/b.vec");
b = lap(a) * x;

In [24]:
norm(lap(a) * x - b) / norm(b)

0.0

In [25]:
la = lap(a);

In [26]:
ts = treeSolver(tree)

F = function(b)
    return ts(b - mean(b))
end

(anonymous function)

In [27]:
@time myx64 = pcg(la, b, F, tol=1e-3, maxits=100000, verbose=true);
norm(lap(a) * myx64 - b) / norm(b)

PCG stopped after: 60 iterations with relative error 0.000723563275623587.
  0.151403 seconds (835.91 k allocations: 35.279 MB, 4.81% gc time)


0.0007235632756237976

In [31]:
dbg = []

for hp in [32,128,1024]
    set_bigfloat_precision(hp)
    laHP = toHighPrecision(la; precision=hp)
    treeHP = toHighPrecision(tree; precision=hp)
    xHP = toHighPrecision(x; precision=hp);
    bHP = toHighPrecision(b; precision=hp);
    tsHP = treeSolver(treeHP)

    FHP = function(b)
        return tsHP(b - mean(b))
    end
    
    push!(dbg, "======= $(hp) bit precision")
#     @time myxHP = pcgVMatNorm(laHP, bHP, FHP, xHP, tol=1e-2, maxits=100000, verbose=true);
    @time myxHP = pcgV(laHP, bHP, FHP, tol=BigFloat(1e-3), maxits=100000, verbose=true);
    norm(laHP * myxHP - bHP) / norm(bHP)
    push!(dbg, "")
    push!(dbg, "")
    
end

PCG stopped after: 64 iterations with relative error 7.9015306778e-04.
  1.672669 seconds (8.53 M allocations: 232.672 MB, 19.00% gc time)
PCG stopped after: 59 iterations with relative error 8.081730050462734633806288754478007505103e-04.
  1.498003 seconds (7.82 M allocations: 230.147 MB, 19.99% gc time)
PCG stopped after: 59 iterations with relative error 8.081730050462734633806288754478007543431900562503488837933403400343950516572309476100377493608583272539722560969008530389697624417272514050422561875361509131388702758736521043161929141638031910547090212151892366730220954500533638430261881891354266399085951152003421492743311451808862501628042730054416333414313e-04.
  2.021308 seconds (7.82 M allocations: 472.580 MB, 22.68% gc time)


In [13]:
# writeToFile("../graphs/se_1/_log_julia.txt", dbg)